<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [3]:
df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
print(x[:3])
print(y[:3])

[[0.00000000e+00 3.00000000e+00 3.00000000e+00 5.00000000e+00
  2.26289017e+00 3.30000000e+01 1.20000000e+01 1.09200000e+02
  1.00000000e+00 2.00793590e+02 5.00000000e+00 2.52000000e+03
  1.85456250e+02 8.45000000e+02 0.00000000e+00 3.00000000e+00
  3.00000000e+00 5.00000000e+00 2.26289017e+00 3.30000000e+01
  1.20000000e+01 1.09200000e+02 1.00000000e+00 2.00793590e+02
  5.00000000e+00 2.52000000e+03 1.85456250e+02 8.45000000e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.02000000e+02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.44700000e+03
  1.48454606e+02 2.36000000e+02 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.02000000e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.44700000e+03 1.48454606e+02 2.36000000e+02
  0.00000000e+00 1.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [6]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[0.00000000e+00 3.00000000e+00 3.00000000e+00 5.00000000e+00
  2.26289017e+00 3.30000000e+01 1.20000000e+01 1.09200000e+02
  1.00000000e+00 2.00793590e+02 5.00000000e+00 2.52000000e+03
  1.85456250e+02 8.45000000e+02 0.00000000e+00 3.00000000e+00
  3.00000000e+00 5.00000000e+00 2.26289017e+00 3.30000000e+01
  1.20000000e+01 1.09200000e+02 1.00000000e+00 2.00793590e+02
  5.00000000e+00 2.52000000e+03 1.85456250e+02 8.45000000e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.02000000e+02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.44700000e+03
  1.48454606e+02 2.36000000e+02 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.02000000e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.44700000e+03 1.48454606e+02 2.36000000e+02
  0.00000000e+00 1.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [8]:
print(mean_train)
print(std_train)

[2.80165503e-01 1.58965606e+00 2.51005948e+00 3.86516680e+00
 7.27548432e-01 4.69955004e+01 2.44943884e+01 7.12848151e+02
 1.89522627e+01 8.41431085e+01 1.87535557e+00 1.75053794e+03
 8.54844961e+01 9.02305949e+02 2.80165503e-01 1.58965606e+00
 2.51005948e+00 3.86516680e+00 7.27548432e-01 4.69955004e+01
 2.44943884e+01 7.12848151e+02 1.89522627e+01 8.41431085e+01
 1.87535557e+00 1.75053794e+03 8.54844961e+01 9.02305949e+02
 2.80165503e-01 1.56038273e-01 3.54797000e-02 4.77527799e-01
 1.54124644e-02 3.53762607e-02]
[4.49079942e-01 1.61874734e+00 6.27510749e+00 1.68820786e+01
 1.11725382e+00 5.20006369e+02 5.15577377e+02 1.07342769e+03
 2.66921932e+01 2.21071813e+02 4.18577891e+00 1.98331660e+03
 8.18967217e+01 1.30622988e+03 4.49079942e-01 1.61874734e+00
 6.27510749e+00 1.68820786e+01 1.11725382e+00 5.20006369e+02
 5.15577377e+02 1.07342769e+03 2.66921932e+01 2.21071813e+02
 4.18577891e+00 1.98331660e+03 8.18967217e+01 1.30622988e+03
 4.49079942e-01 3.62891623e-01 1.84988894e-01 4.99494

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [10]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[-0.62386555  0.87125637  0.07807683  0.06722118  1.37421033 -0.02691409
  -0.02423378 -0.56235568 -0.67256604  0.52765877  0.74649056  0.38796734
   1.2207052  -0.04387126 -0.62386555  0.87125637  0.07807683  0.06722118
   1.37421033 -0.02691409 -0.02423378 -0.56235568 -0.67256604  0.52765877
   0.74649056  0.38796734  1.2207052  -0.04387126 -0.62386555 -0.42998588
  -0.19179368  1.0460014  -0.12511486 -0.19150362]
 [-0.62386555 -0.36426689 -0.24064281 -0.16971647 -0.65119351  0.10577659
  -0.04750866 -0.66408586 -0.71003018 -0.38061437 -0.44803025 -0.15304563
   0.76889659 -0.51009854 -0.62386555 -0.36426689 -0.24064281 -0.16971647
  -0.65119351  0.10577659 -0.04750866 -0.66408586 -0.71003018 -0.38061437
  -0.44803025 -0.15304563  0.76889659 -0.51009854 -0.62386555  2.32565778
  -0.19179368 -0.95602167 -0.12511486 -0.19150362]
 [-0.62386555 -0.36426689 -0.24064281 -0.16971647 -0.65119351 -0.07883654
  -0.04750866 -0.23368891 -0.59763777 -0.35362669 -0.20912609  0.32141216
   0.68953

In [0]:
def create_sampler(y):
  _, counts = np.unique(y, return_counts=True)
  n_samples = len(y)
  class_weights = n_samples / counts
  weights = class_weights[y]
  return data.WeightedRandomSampler(weights=weights, num_samples=n_samples, replacement=True)

In [0]:
sampler = create_sampler(y_train)

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, sampler=sampler)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [15]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[-0.6239,  0.2535, -0.0813, -0.1105,  0.2439, -0.0865, -0.0436, -0.6492,
         -0.6726, -0.3802,  0.0298,  1.3651, -0.3635,  0.0151, -0.6239,  0.2535,
         -0.0813, -0.1105,  0.2439, -0.0865, -0.0436, -0.6492, -0.6726, -0.3802,
          0.0298,  1.3651, -0.3635,  0.0151, -0.6239, -0.4300, -0.1918,  1.0460,
         -0.1251, -0.1915],
        [-0.6239, -0.3643, -0.2406, -0.1697, -0.6512, -0.0750, -0.0359,  4.4029,
          2.1747, -0.3791, -0.2091,  0.7560,  0.3707,  0.2340, -0.6239, -0.3643,
         -0.2406, -0.1697, -0.6512, -0.0750, -0.0359,  4.4029,  2.1747, -0.3791,
         -0.2091,  0.7560,  0.3707,  0.2340, -0.6239, -0.4300, -0.1918,  1.0460,
         -0.1251, -0.1915],
        [-0.6239, -0.3643, -0.2406, -0.1697, -0.6512, -0.0865, -0.0436, -0.2039,
         -0.6726, -0.3806, -0.2091,  0.4152,  0.5419,  0.4530, -0.6239, -0.3643,
         -0.2406, -0.1697, -0.6512, -0.0865, -0.0436, -0.2039, -0.6726, -0.3806,
         -0.2091,  0.4152,  0.5419,  0.4530, -0.6239,

In [0]:
class Classifier(nn.Module):
  def __init__(self, epochs, input_size, hidden_size, val_loss=None):
    super(Classifier, self).__init__()
    self.epochs = epochs
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.val_loss = val_loss
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fcout(x))
    return x

In [17]:
classifier = Classifier(epochs=100, input_size=x.shape[1], hidden_size=512)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fcout): Linear(in_features=512, out_features=1, bias=True)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'epochs': classifier.epochs,
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'val_loss': classifier.val_loss,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['epochs'], checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['val_loss'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [20]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.5125],
        [0.4879],
        [0.4966]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [22]:
train_loss = 0
epochs = classifier.epochs
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  with torch.no_grad():
    classifier.eval()
    val_loss = 0
    for inputs, targets in val_dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      loss = criterion(outputs.squeeze(), targets.float())
      val_loss += loss.item()

    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    print('Epoch: {}, Train loss: {}, Val loss: {}'.format(epoch, train_loss, val_loss))

    if classifier.val_loss is None or val_loss < classifier.val_loss:
      classifier.val_loss = val_loss
      classifier.epochs = epoch
      save_classifier(classifier)

Epoch: 0, Train loss: 0.5700377304303018, Val loss: 0.5830285835422968
Epoch: 1, Train loss: 0.5449244233049514, Val loss: 0.5026258332164664
Epoch: 2, Train loss: 0.5208037640757665, Val loss: 0.5248800112228644
Epoch: 3, Train loss: 0.5101704248370499, Val loss: 0.5450558878089252
Epoch: 4, Train loss: 0.4981757103052625, Val loss: 0.5222430684064564
Epoch: 5, Train loss: 0.4846446341554504, Val loss: 0.4865446031877869
Epoch: 6, Train loss: 0.4818894137057301, Val loss: 0.5440967796664489
Epoch: 7, Train loss: 0.4803206242728459, Val loss: 0.5336525589227676
Epoch: 8, Train loss: 0.481710764707202, Val loss: 0.4338073487344541
Epoch: 9, Train loss: 0.47132879111173576, Val loss: 0.5052180478447362
Epoch: 10, Train loss: 0.4789154403076368, Val loss: 0.45737868978788976
Epoch: 11, Train loss: 0.4661008955424967, Val loss: 0.5512586832046509
Epoch: 12, Train loss: 0.46405875196944707, Val loss: 0.46904259822086286
Epoch: 13, Train loss: 0.4575485185862871, Val loss: 0.4802190661430359

In [0]:
def calculate_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  return np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=1))

def gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calculate_recalls(targets, predictions)

    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls)    

In [24]:
classifier = load_classifier()
if torch.cuda.is_available():
  classifier = classifier.cuda()
train_accuracy = gmean(classifier, train_dataloader)
test_accuracy = gmean(classifier, test_dataloader)
print('Epochs: {}'.format(classifier.epochs))
print('Train g-mean: {}'.format(train_accuracy.item()))
print('Test g-mean: {}'.format(test_accuracy.item()))

Epochs: 8
Train g-mean: 0.7614146313143798
Test g-mean: 0.7613957045073717
